In [1]:
# Importation des librairies
from webscraping.ws_fonctions import *


In [7]:
get_comparaison_notes(2021, 5, 'action')

,date,durée,spectateur,presse,genre1,genre2,genre3
0,20 octobre 2021,1h 38min,"2,1","1,7",Action,Fantastique,
1,15 décembre 2021,2h 28min,"3,9","3,3",Action,Aventure,Fantastique
2,6 octobre 2021,2h 43min,"3,7","3,1",Action,Espionnage,Thriller
3,28 juillet 2021,2h 12min,"3,3","3,7",Action,Aventure,Fantastique
4,16 juin 2021,1h 59min,"3,6","3,0",Action,Thriller,
5,29 décembre 2021,2h 11min,"3,4","3,2",Action,Espionnage,
6,9 juillet 2021,1h 37min,"2,8","3,3",Action,Aventure,Drame
7,12 novembre 2021,1h 58min,"3,1","2,3",Action,Aventure,Policier
8,2 juin 2021,1h 32min,"3,7","2,8",Action,,
9,1 décembre 2021,2h 04min,"3,4","3,0",Action,Aventure,Comédie
